# 9. Exporting Data Example

This notebook shows all the different ways to export data from memory. This first example exports all conversations from Azure SQL memory with their respective score values in a JSON format. Without using the database query editor, these export functions allow for a quick approach to exporting data from memory.

In [ ]:
from pyrit.memory.azure_sql_memory import AzureSQLMemory
from pyrit.common import default_values
from pathlib import Path

default_values.load_environment_files()

memory = AzureSQLMemory()

# Define file path for export
json_file_path = Path("conversation_and_scores_json_example")
# csv_file_path = Path(\"conversation_and_scores_csv_example\")

# Export the data to a JSON file
conversation_with_scores = memory.export_all_conversations_with_scores(file_path=json_file_path, export_type="json")
print(f"Exported conversation with scores to JSON: {json_file_path}")

# Export the data to a CSV file
# conversation_with_scores = memory.export_all_conversations_with_scores(file_path=json_file_path, export_type="csv")
# print(f"Exported conversation with scores to CSV: {csv_file_path}")

Error fetching data from table ScoreEntries: (pyodbc.Error) ('07002', '[07002] [Microsoft][ODBC Driver 18 for SQL Server]COUNT field incorrect or syntax error (0) (SQLExecDirectW)')
[SQL: SELECT [ScoreEntries].id AS [ScoreEntries_id], [ScoreEntries].score_value AS [ScoreEntries_score_value], [ScoreEntries].score_value_description AS [ScoreEntries_score_value_description], [ScoreEntries].score_type AS [ScoreEntries_score_type], [ScoreEntries].score_category AS [ScoreEntries_score_category], [ScoreEntries].score_rationale AS [ScoreEntries_score_rationale], [ScoreEntries].score_metadata AS [ScoreEntries_score_metadata], [ScoreEntries].scorer_class_identifier AS [ScoreEntries_scorer_class_identifier], [ScoreEntries].prompt_request_response_id AS [ScoreEntries_prompt_request_response_id], [ScoreEntries].timestamp AS [ScoreEntries_timestamp], [ScoreEntries].task AS [ScoreEntries_task] 
FROM [ScoreEntries] 
WHERE [ScoreEntries].prompt_request_response_id IN (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?

Exported conversation with scores to JSON: conversation_and_scores_json_example


## Importing Data as NumPy DataFrame

You can use the exported JSON or CSV files to import the data as a NumPy DataFrame. This can be useful for various data manipulation and analysis tasks.

In [ ]:
import pandas as pd  # type: ignore

df = pd.read_json(json_file_path)
df.head()

""
